In [1]:
import cv2
import os
import numpy as np

In [2]:
def dark_channel_prior(img, window_size=15):
    # Step 1: Median Filter
    median_filtered = cv2.medianBlur(img, window_size)

    # Step 2: RGB Separation
    R, G, B = cv2.split(median_filtered)

    # Step 3: Dark Channel Prior
    dark_channel = cv2.min(R, cv2.min(G, B))
    
    return dark_channel

In [3]:
def estimate_transmission(img, dark_channel, omega=0.95, window_size=15):
    A = dark_channel_prior(img, window_size)
    transmission = 1 - omega * A
    return transmission

In [4]:
def guided_filter(p, I, radius=40, eps=1e-3):
    mean_p = cv2.boxFilter(p, -1, (radius, radius))
    mean_I = cv2.boxFilter(I, -1, (radius, radius))
    mean_Ip = cv2.boxFilter(I * p, -1, (radius, radius))
    cov_Ip = mean_Ip - mean_I * mean_p
    mean_II = cv2.boxFilter(I * I, -1, (radius, radius))
    var_I = mean_II - mean_I * mean_I
    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I
    mean_a = cv2.boxFilter(a, -1, (radius, radius))
    mean_b = cv2.boxFilter(b, -1, (radius, radius))
    q = mean_a * I + mean_b
    return q

In [5]:
def dehaze(input_image, window_size=15, omega=0.95, guided_filter_radius=40, guided_filter_eps=1e-3):
    # Step 1: Dark Channel Prior
    dark_channel = dark_channel_prior(input_image, window_size)

    # Step 2: Estimate Transmission Map
    transmission = estimate_transmission(input_image, dark_channel, omega, window_size)

    # Step 3: Guided Filter
    dehazed_image = guided_filter(transmission, input_image, guided_filter_radius, guided_filter_eps)

    return dehazed_image

In [6]:
# Get the current directory
current_directory = os.getcwd()

# Define input and output folder paths
input_folder = os.path.join(current_directory, 'Input')
output_folder = os.path.join(input_folder, 'Output')

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

for image_file in image_files:
    # Load the underwater image
    input_image_path = os.path.join(input_folder, image_file)
    input_image = cv2.imread(input_image_path)

    # Enhance the image using the Dark Channel Prior
    enhanced_image = dehaze(input_image)

    # Save the enhanced image to the output folder within the input folder
    output_image_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_image_path, enhanced_image)

print("Enhancement complete. Enhanced images are saved in the 'Output' folder within the 'Input' folder.")

ValueError: operands could not be broadcast together with shapes (720,1280,3) (720,1280) 